# Introduction
The first project that you will complete this semester is to design, implement, and test a controller that uses a single-gimbal control moment gyroscope (CMG) to reorient a platform in a gravitational field, as pictured below:

# System Dynamics and Equations of Motion
This is the model of the cart. There are two generalized coordinates: $\theta$ (the pendulum angle) and $\phi$ (the wheel angle). Because we assume that all the wheels are rolling without slipping, each wheel has the same angle. Further, the $x$ coordinate of the cart can be expressed in terms of the wheel angle and the radius of the wheels, $r$. Lastly, the input to the system is some torque, $\tau$, that is applied equally to all the wheels.

The first step in the Lagrangian mechanics approach is to calculate the total kinetic energy of the system and the total potential energy of the system with respect to the generalized coordinates and their derivatives. We can do this using the Python package Sympy.

In [ ]:
# Import everything we need from SymPy
from sympy import Symbol, Matrix, Function, Derivative, N
from sympy import diff, simplify, sin, cos, solve, init_printing
init_printing() # This function will make the outputs of SymPy look prettier and be easier to read

Now we are going to define the parameters of the system as constants:

In [ ]:
# Constants of the system
mp = 1.0
mc = 6.0
mw = 1.0
l = 1.0
r = 0.25
g = 9.81

Now we make symbols and functions. These are elements of SymPy and can be thought of as exactly the same as symbols (variables) and functions from math.

In [ ]:
# Time is a symbol (variable)
t = Symbol('t')

# The generalized coordinates and the input torque are both functions of time.
# This means that they are initialized as Functions.
theta = Function('theta')
phi = Function('phi')
tau = Function('tau')

Now we will start to calculate the energies of the system. We can start by getting the kinetic energy of the mass on top of the rod. We can do this by:
1. Defining its position in terms of the generalized coordinates
2. Take the derivate of the position with respect to time to get the velocity
3. Calculate the kinetic energy from the velocity
4. Calculate the potential energy by observation

In [ ]:
# Get the position of the mass in terms of phi and theta
pos = Matrix([-r*phi(t) - l*sin( theta(t) ),
              0.0,
              l*cos( theta(t) )])

In [ ]:
# Take the derivative of the position with respect to time
vel = diff(pos,t)

In [ ]:
# Get the kinetic energy of the mass from the velocity
mass_KE = 0.5 * mp * (vel.T @ vel)[0,0]

In [ ]:
# Get the potential energy of the mass based on its position
mass_PE = mp*g*l*cos(theta(t))

Because the rod is assumed to be massless and have no inertia, we can move on the the energy of the cart. The cart does not change height, so its potential energy is automatically 0. This means all we have to do is calculate the kinetic energy.

In [ ]:
# Get the position of the cart in terms of phi and theta
pos = Matrix([-phi(t)*r,
              0.0,
              0.0])

In [ ]:
# Take the derivative of the position with respect to time
vel = diff(pos,t)

In [ ]:
# Get the kinetic energy of the cart from the velocity
cart_KE = 0.5 * mc * (vel.T @ vel)[0,0]

Next up, the wheels. Similarly, they have no potential energy, however they do have rotational energy. So we need to calculate both thier kinetic and rotational energy. Also, while calculating the position, we will ignore the constant offsets that each of the four wheels have. We can do this because constants disappear during differentiation.

In [ ]:
# Get the position of the wheels in terms of phi and theta
pos = Matrix([-phi(t)*r,
              0.0,
              0.0])

In [ ]:
# Take the derivative of the position with respect to time
vel = diff(pos,t)

In [ ]:
# Get the kinetic energy of all four wheels from the velocity
wheel_KE = 4.0 * (0.5 * mw * (vel.T @ vel)[0,0])

In [ ]:
# Get the rotational kinetic energy of all four wheels
I = 0.5*mw*r**2
wheel_RE = 4.0*(I*diff(phi(t),t)**2)

Now we calculate the lagrangian of the system via the formula:$$L=T-V$$ where $T$ is the total kinetic energy and $V$ is the total potential energy.

In [ ]:
# Get the lagrangian
L = (mass_KE + cart_KE + wheel_KE + wheel_RE) - mass_PE
print("System Lagrangian:")
L = simplify(L)
N(L, 3) # This rounds floating point number to 3 places and then prints to the screen

Finally, we get the equations of motion of the system via the formulas:
$$\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\theta}} \right) - \frac{\partial L}{\partial \theta}=0$$
$$\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\phi}} \right) - \frac{\partial L}{\partial \phi}=\tau$$

In [ ]:
# Get the first equation of motion
eq1 = diff(diff(L, Derivative(theta(t), t)), t) - diff(L, theta(t))

In [ ]:
# Get the second equation of motion
eq2 = diff(diff(L, Derivative(phi(t), t)), t) - diff(L, phi(t)) - tau(t)

This form of equations of motion is not very helpful to us right now, so next we will place them in standard form. A standard form system of ordinary differential equations satisfy the following requirements
* All equations are exactly first order ordinary differential equations
* Each equation only has exactly one time derivative variable in it
* Each equation has the form $\frac{d x}{dt} = f(x,y,z,...)$

Because the equations of motion above are second order, we need to apply a trick to place them in standard form. Namely, we introduce new variables that increase the number of equations while reducing the order of each equation. For example, let's introduce two new variables, $\omega_{\theta}$ and $\omega_{\phi}$ that satisfy the standard form ordinary differential equations:
$$\frac{d \theta}{dt} = \omega_{\theta}$$
$$\frac{d \phi}{dt} = \omega_{\phi}$$
Then, by taking the derivative of both sides, we see that
$$\frac{d^2 \theta}{d t^2} = \frac{d \omega_{\theta}}{dt}$$
$$\frac{d^2 \phi}{d t^2} = \frac{d \omega_{\phi}}{dt}$$
Let's make this change of variables to our equations of motion right now.

In [ ]:
# Make the new functions for our change of variables
omega_theta = Function('omega_theta')
omega_phi = Function('omega_phi')

In [ ]:
# Make the change of variables for equation 1
eq1 = eq1.subs({Derivative(theta(t), (t, 2)) : Derivative(omega_theta(t), t), 
                Derivative(phi(t), (t, 2))   : Derivative(omega_phi(t), t),
                Derivative(theta(t), t)      : omega_theta(t),
                Derivative(phi(t), t)        : omega_phi(t)})

In [ ]:
# Repeat for equation 2
eq2 = eq2.subs({Derivative(theta(t), (t, 2)) : Derivative(omega_theta(t), t), 
                Derivative(phi(t), (t, 2))   : Derivative(omega_phi(t), t),
                Derivative(theta(t), t)      : omega_theta(t),
                Derivative(phi(t), t)        : omega_phi(t)})

The final step to place the equations in standard form is to ensure they have the form $\frac{d x}{dt} = f(x,y,z,...)$. We do this now.

In [ ]:
# Solve the first two equations of motion for \dot{omega_theta} and \dot{omega_phi}
soln = solve([eq1, eq2],
             Derivative(omega_theta(t), t),
             Derivative(omega_phi(t), t))
soln = simplify(soln)

We now have a system of ordinary differential equations in standard from. Specifically, we have:
$$\frac{d \omega_{\theta}}{dt} = f_{\omega_{\theta}}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$
$$\frac{d \omega_{\phi}}{dt} = f_{\omega_{\phi}}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$
$$\frac{d \theta}{dt} = f_{\theta}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$
$$\frac{d \phi}{dt} = f_{\phi}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$

Let's now combine the right hand side of this system into a single vector.

In [ ]:
sys = Matrix([soln[Derivative(omega_theta(t), t)],
              soln[Derivative(omega_phi(t), t)],
              omega_theta(t),
              omega_phi(t)])
sys = simplify(sys)
N(sys, 3) # This rounds floating point number to 3 places and then prints to the screen

This is our complete system model!

# System Linearization and State Space Form

In [ ]:
# FIXME: Linearize the system here

# Gain Selection and Controller Design

In [ ]:
# FIXME: Select the control gains here

In [ ]:
# This module is part of condynsate and let's you detect key presses
from condynsate.keyboard import Keys

# Create the Controller class
class Controller():
    def __init__(self):
        """
        Initializes the controller. You should initialize any member variables that
        the controller will use to their starting values. This function is called
        when a new instance of the Controller class is created.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # FIXME: Replace the following manual controller initialization with your own initialization
        # Create a keyboard press tracker for a manual controller
        self.keys = Keys() 

    
    def reset(self):
        """
        Resets the controller to an initial state. You should reset any variables
        that track states, times, etc. in this function. This function will be 
        called just before the simulation starts running and any time the simulation
        is reset.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # FIXME: Replace the following manual controller reset with your own reset function
        pass 

    
    def run(self, **kwargs):
        """
        Runs the controller. Data is passed from the simulation environment to the
        controller via kwargs, the controller calculates the inputs to the system,
        and then returns them via a return statement
    
        Parameters
        ----------
        **kwargs : Dictionary
            A dictionary of data passed from the simulation environment to the 
            controller. Specific data is extracted from kwargs via the statement:
            kwargs["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys
            of kwargs for this project are as follows:

            pendulum_angle : Float
                The current angle of the pendulum in radians (theta)

            wheel_angle : Float
                The current target angle of the wheels in radians (phi)

            pendulum_velocity : Float
                The current angular velocity of the pendulum in radians/second

            wheel_velocity : Float
                The current target angular velocity of the wheels in radians/second

            time : Float
                The current simulation time in seconds
            
    
        Returns
        -------
        inputs : List with length equal to the number of inputs
            This is a list of the inputs calculated by the controller. In this
            project, there is only one input, the torque to be applied to the 
            wheels. Even though there is only one input in this project,
            it still must be placed in a list of length one.
        
        """
        # FIXME: Replace the following manual controller code with your controller code
        # Detect key presses and set torque accordingly 
        if self.keys.is_pressed("shift+d"):
            torque = 5.0
        elif self.keys.is_pressed("shift+a"):
            torque = -5.0
        elif self.keys.is_pressed("d"):
            torque = 1.0
        elif self.keys.is_pressed("a"):
            torque = -1.0
        else:
            torque = 0.0
    
        # Return the manually set torque
        inputs = [torque]
        return inputs

In [ ]:
# Import the cart project. This module is used to simulate, render, and plot a wheel dynamic system
import ae353_cart

In [ ]:
# Create an instance of the cart simulator
sim = ae353_cart.Cart_sim()

In [ ]:
# Create an instance of our Controller class
controller = Controller()

## Running the simulation
Now we are ready to run the simulation and collect data. To do this we call the ``sim.run()`` function. This function takes an instance of the ``Controller`` class as an argument and returns ``data``. The parameters and return values of ``sim.run()`` are shown below.

#### Parameters

> **controller** : *member of Controller class*  
> Your controller that will generated inputs to the system.
> 
> **initial_pendulum_angle** : *Float, optional*  
> The initial angle of the pendulum in radians. This is set when the simulation starts and when the simulation is reset. The default value is 0.0.
> 
> **initial_wheel_angle** : *Float, optional*  
> The initial angle of the wheels in radians. This is set when the simulation starts and when the simulation is reset. The default value is 0.0.
> 
> **initial_pendulum_velocity** : *Float, optional*  
> The initial velocity of the pendulum in radians/second. This is set when the simulation starts and when the simulation is reset. The default value is 0.0.
> 
> **initial_wheel_velocity** : *Float, optional*  
> The initial velocity of the wheels in radians/second. This is set when the simulation starts and when the simulation is reset. The default value is 0.0.


#### Returns

> **data** : *Dictionary of Lists*  
> A dictionary containing all relevant data generated during the simulation. Specific data is extracted via the statement: data["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys of data for this project are as follows:
> 
> **data["pendulum_angle"]** : *List of Floats*  
> A list of the pendulum angle in radians at each time stamp during the simulation.
> 
> **data["wheel_angle"]** : *List of Floats*  
> A list of the wheel angle in radians at each time stamp during the simulation.
> 
> **data["pendulum_velocity"]** : *List of Floats*  
> A list of the pendulum velocity in radians/secondat each time stamp during the simulation.
> 
> **data["wheel_velocity"]** : *List of Floats*  
> A list of the wheel velocity in radians/second at each time stamp during the simulation.
> 
> **data["torque"]** : *List of Floats*  
> A list of the applied torque in Newton-meters at each time stamp during the simulation.
> 
> **data["time"]** : *List of Floats*  
> A list of the time stamps in seconds.

You can use the keyboard to interact with the simulation:
* press **ENTER** to start the simulation
* press **BACKSPACE** to reset the simulation
* press **SPACE** to pause the simulation
* press **ESC** to end the simulation

If the manual controller is being used:
* press **a** to apply small negative torque to wheels
* press **SHIFT+a** to apply large negative torque to wheels
* press **d** to apply small positive torque to wheels
* press **SHIFT+d** to apply large positive torque to wheels 

In [ ]:
# Run the simulation and collect the simulation data
data = sim.run(controller)

## Post-processing the simulation data
After the simulation is run, we can plot the data using the Matplotlib package. To start, we import matplotlib.

In [ ]:
# Import the pyplot module from the matplotlib package
import matplotlib.pyplot as plt

Now we can make whichever plots we see fit to make sense of the simulation data. Here we plot the pendulum versus time as well as the applied torque versus time.

In [ ]:
# Create a figure that has two plots in it
fig, axes = plt.subplots(2, 1, figsize=(7.2, 5.4), sharex=True)

In [ ]:
# Create a plot of the wheel angle and target wheel angle versus time on the first plot
axes[0].plot(data['time'], data['pendulum_angle'], label="Angle", c='k', lw=2.5)
axes[0].set_ylabel("Angle [rad]", fontsize=12)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].legend(fontsize=12, shadow=True)
axes[0].set_title("Pendulum Angle vs. Time", fontsize=14)
axes[0].grid()

In [ ]:
# Create a plot of the torque versus time on the second plot
axes[1].plot(data['time'], data['torque'], label="Torque", c='b', lw=2.5)
axes[1].set_xlabel("Time [s]", fontsize=12)
axes[1].set_ylabel("Torque [Nm]", fontsize=12)
axes[1].tick_params(axis='both', labelsize=12)
axes[1].set_title("Torque vs. Time", fontsize=14)
axes[1].grid()

In [ ]:
# Show the figure
fig.tight_layout(pad=2.0)
plt.show()